# Bash

Когда речь заходит об обработке больших данных, на первый план выходят задачи по грамотной манипуляции данными, конвертированию и выгрузке, настройке специализированных инструментов и так далее. Чтобы уметь решать такие задачи с кайфом, нужно обрести уверенные навыки работы в терминале и командных оболочках.

Bash - наиболее популярная командная оболочка в UNIX-like система. Когда вы пользуетесь терминалом на своем компьютере, то вы почти наверное используете именно Bash.

В баш встроено огромное количество функциональности, которая не ограничивается лишь запуском программ.

# Table of Contents 


1. Потоки данных, pipe, xargs, shasum
2. Head, Tail
3. Sort, shuf, uniq
4. Wc, cut, grep, awk, sed
5. Archives
6. HDFS
7. Bash scripts

# Потоки данных

Сильная сторона языка Bash - это возможности по работе с перенаправлениями потоками данных между подпрограммами.

Для Bash каждую программу можно представить в следующем виде - это черный ящик, которому

На вход подается какой-то источник данных, а также аргументы запуска

На выход выдает два источника данных - это вывод результата и вывод ошибок.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/bash-flow-control-1.png">

Чаще всего к такому "ящику" подключаются три стандартный источника данных - это stdin, stdout и stderr.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/bash-flow-control-2.png">

Все эти три источника, как и все в UNIX, являются виртуальными файлами и выполняют следующие функции

* stdin представляет весь поток данных, который пользователь вводит с клавиатуры
* stdout представляет весь поток данных, который программа печатает на экран
* stderr представляет весь поток данных об ошибках в работе программы, который программа также печатает на экран

Все эти источники находятся по следующим путям: `/dev/stdout`, `/dev/stdin`, `/dev/stderr`. Можно заглянуть в директорию `/dev/` и посмотреть, сколько еще виртуальных источников данных есть в компьютере.

In [ ]:
! mkdir -p tempsem2

In [ ]:
%cd tempsem2

In [ ]:
! ls /dev

Волшебство UNIX заключается в том, что мы можем работать с этими источниками как с самыми обычными файлами. Например, давайте попробуем что-то написать в файл /dev/stdout

In [ ]:
%%writefile hello-stdout.py
with open('/dev/stdout', 'w') as f:
    f.write("HELLO, STDOUT!")  # Пишем в специальный файл, вместо print

In [ ]:
! python3 hello-stdout.py

Вместо стандартных потоков ввода\вывода можно подставлять произвольные источники данных.

**Оператор >** позволяет перенаправлять стандартный вывод в любой другой файл. Попробуем, например, написать что-то в новый файл используя команду echo.

In [ ]:
! echo "dsjfhkdsfh"

In [ ]:
! echo "message from echo" > file.txt

In [ ]:
! cat file.txt

In [ ]:
! cat file.txt file.txt # Читает файл два раза

In [ ]:
! cat file.txt file.txt > doubled-file.txt

In [ ]:
! cat doubled-file.txt

Если файл уже существует, то этот оператор полностью перезатрет его содержимое

In [ ]:
! echo "new message" > doubled-file.txt

In [ ]:
! cat doubled-file.txt

**Оператор >>** позволяет не перезаписывать целиком файл, а лишь добавить в конец новые данные

In [ ]:
! cat file.txt file.txt > doubled-file.txt

In [ ]:
! cat doubled-file.txt

In [ ]:
! echo "new message" >> doubled-file.txt

In [ ]:
! cat doubled-file.txt

Помимо вывода программы, можно поменять и ее ввод.

**Оператор <** позволяет поменить стандартный ввод программы на другой файл, делая вид для программы, будто бы пользователь ввел эти данные с клавиатуры.

Напишем простенькую программу, которая читает пользовательский ввод с клавиатуры и потестируем этот оператор.

In [ ]:
%%writefile repeat.py

data = input()
for i in range(3):
    print(data)

In [ ]:
! cat file.txt

In [ ]:
! python3 repeat.py < file.txt

In [ ]:
! python3 -c "import sys; print(list(sys.stdin))" < doubled-file.txt

Если хочется направить на stdin несколько строк, но при этом не из файла, а прямо из скрипта, то можно воспользваться **оператором <<**. Для него нужно отдельно указать маркер начала и конца данных.

In [ ]:
! python3 -c "print(2 ** 3)"

In [ ]:
%%bash

python3 -c "import sys; print(list(sys.stdin))" <<BUBA
big
multiline
message
from
script
BUBA

Если хочется направить ровно одну строку, то можно воспользоваться **оператором <<<**. Он подаст на stdin ту строку, которую мы передадим в качестве аргумента.

In [ ]:
%%bash

python3 -c "import sys; print(list(sys.stdin))" <<< "one line message from script"

Комбинирование этих возможностей может позволить решить некоторые задачи. Например генерировать какой-то файл на лету.

In [ ]:
%%bash

cat > message2.txt <<BIBA 
This file was created directly from script
Using amazing features of Bash
BIBA

In [ ]:
! cat message2.txt

Или например решать проблему с интерактивными программами, которые хотят какого-то действия от пользователя.

In [ ]:
%%writefile interactive.py

answer = input("Are you sure you want to do X? (y/n)")
if answer == 'y':
    print("DOING X")
else:
    print("Canceling")

In [ ]:
! python3 interactive.py <<< "y"

In [ ]:
! python3 interactive.py <<< "n"

Возможности перенаправлений не ограничиваются только статическими текстовыми файлами. 

**Оператор <()** позворяет представить вывод программы как специальный файл дескриптор (чаще всего он выглядит как `/dev/fd/63`) . Основная особенность этого оператора от обычного **оператора >** в том, что не используется дополнительное место на диске. Читая из этого специального дескриптора мы напрямую получаем доступ к выводу программы без необходимости сохранять его на диске в явном виде.

Сам оператор при применении запускает переданную программу, создает специальный файл и возращает путь до этого файла.

Этим можно воспользоваться, чтобы компоновать работу нескольких программ.

In [ ]:
! echo "biba" > file
! cat file

In [ ]:
! echo <(echo PIPA)  # Напечатали путь до файла, в коротый подключен вывод программы echo PIPA

In [ ]:
! cat <(echo PIPA)

In [ ]:
%%writefile read-file.py

import sys

print("Path to file = {}".format(sys.argv[1]))

with open(sys.argv[1], 'r') as f:
    content = f.read()
    print(content)

In [ ]:
! cat file.txt

In [ ]:
! python3 read-file.py file.txt

In [ ]:
! python3 read-file.py <(echo PIPA)

Комбинируя возможности операторов **<** и **<()** можно перенаправлять вывод одной команды в другой.

Наглядный пример - команда `yes`. Она решает уже рассмотренную проблему работы с интерактивными программами, без конца печатая символ `y` на стандартный вывод. 

(Интересный факт - люди любят соревноваться в "производительности" программы `yes`. Согласно [треду на Реддите](https://www.reddit.com/r/unix/comments/6gxduc/how_is_gnu_yes_so_fast/), рекорд - вывод `y` со скоростью 123 Гигабита в секунду. Зачем нужная такая производительность в команде `yes`? Ну чтобы было смешно.)

In [ ]:
!cat interactive.py

In [ ]:
! python3 interactive.py < <(yes)

Другой полезный пример - сравнение двух каталогов. Сущестует программа `diff`, которая получает два файла и выводит построчные различия в этий файлах. Можно скомбинировать ее с командой `ls`, чтобы научиться сравнивать директории.

In [ ]:
! mkdir -p folder1
! touch folder1/file1.txt folder1/file2.txt folder1/file3.txt

! mkdir -p folder2
! touch folder2/file2.txt folder2/file3.txt folder2/file4.txt

In [ ]:
! ls folder1

In [ ]:
! ls folder2

In [ ]:
! diff <(ls folder1) <(ls folder2)

Идея соединять процессы через stdin\stdout очень популярна и для этого есть более удобный интерфейс - pipes или **оператор |**

Все команды, соединенные через | запускаются одновременно и общаются друг с другом через stdin\stdout

In [ ]:
! echo hello | cat

In [ ]:
! echo hi | wc -l  # wc считает сколько строк в входных данных

In [ ]:
! cat doubled-file.txt | wc -l

In [ ]:
! ls -l | wc -l

In [ ]:
! ls -l | cat | wc -l | python3 -c "print(int(input()) * 2)"

In [ ]:
! yes | python3 interactive.py

In [ ]:
! yes | head  # Смотрим первые 10 строк файла

Существует также родственный оператор **$()**. Он также запускает переданную программу, однако перенаправляет вывод не в файл, а прямо в bash. То есть вывод программы можно использовать как строку внутри скрипта.

In [ ]:
! pwd

In [ ]:
! echo I am here - $(pwd)

In [ ]:
%%writefile file-to-read.txt
/etc/hosts

In [ ]:
! cat $(cat file-to-read.txt)

### xargs


xargs это команда, которая позволяет взять данные из стандартного входа, разбить их и дать как список аргументов для некоторой другой команды. Звучит странно, посмотрим на примеры:

In [ ]:
# Посмотрим мета-информацию только об определнных файлах в папке
# создаем 2 txt и 2 других формата

! mkdir -p folder1
! touch folder1/file1.bin folder1/file2.txt folder1/file3.json folder1/file4.txt


In [ ]:
# по сути xargs применил ls -la к каждому файлу в отдельности
!find folder1 -name "*.txt"| xargs ls -la 

In [ ]:
!find folder1 -name "*.txt"| xargs wc -l

In [ ]:
%%file folder1/file2.txt
pam 1
pam 2
smth
smth

In [ ]:
%%file folder1/file4.txt
pam 3
pam 4
smth
smth

In [ ]:
# по сути xargs применил grep к каждому файлу в отдельности
!find folder1 -name "*.txt"| xargs grep "pam"

## shasum
А как быстро сравнить два больших файла? Shasum посчитает хэш, по которому потом удобно сравнивать файлы. ЭТО БЫСТРО даже для очень больших файлов

In [ ]:
%%file folder1/file_new.txt
pam 1
pam 2
smth
smth

In [ ]:
!find folder1 -name "*.txt"| xargs shasum

# Полезные программы

Bash - всего лишь оболочка и не умеет самостоятельно решать какие-то задачи. Основную работу выполняют установленные программы, которые можно вызывать из bash. Их можно использовать гораздо эффективнее, используя их вместе с возможностями оболочки.

### Head
head читает определенное количество данных с начала файла. Это полезно, например, когда хочется посмотреть на часть данных, которая лежит на диске.

In [ ]:
! head /etc/hosts

In [ ]:
! head -n 2 /etc/hosts  # Читаем только первые 2 строки

In [ ]:
! head -c 10 /etc/hosts  # Читаем только первые 10 байт

In [ ]:
! cat /etc/hosts | head -n 2  # Как и почти все программы, которые мы рассмотрим, умеет работать с вводом

### Tail
tail делает то же самое, что и head, но с конца файла

In [ ]:
! tail /etc/hosts

In [ ]:
! tail -n 2 /etc/hosts

In [ ]:
! tail -n +3 /etc/hosts  # все строки после второй строки (включая вторую строку)

In [ ]:
! cat /etc/hosts | tail -n 2

### Sort
sort сортирует входные данные. По умолчанию используется лексикографический порядок, но это поведение можно поменять с использованием специальных опций.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=sort&category=1

In [ ]:
%%writefile numbers.txt
3
5
1
2
6
5
9
4
5
6
7
3
2
10
5
6

In [ ]:
! sort numbers.txt  # Сортируем как строки

In [ ]:
! cat numbers.txt | sort

In [ ]:
! cat numbers.txt | sort -n # Сортируем как числа

Если в данных есть сразу несколько "колонок" (например в каждой строке есть значения, разделенные пробельным символом), то можно отдельно указать, по какому полю необходимо сортировать

In [ ]:
%%writefile number-table.txt
1 10
2 9
3 8
4 7
5 6
6 5
7 4
8 3
9 2
10 1

In [ ]:
! cat number-table.txt | sort -k1,1 -n

In [ ]:
! cat number-table.txt | sort -k2,2 -n

In [ ]:
! cat number-table.txt | sort -k2,2 -k1,1 -n

In [ ]:
! cat numbers.txt | sort -n -r  # Сортируем в обратном порядке

### Shuf
shuf напротив, случайным образом перемешивает входящие данные. НА МАКЕ: ```brew install coreutils```

In [ ]:
! cat numbers.txt | shuf

In [ ]:
! cat numbers.txt | shuf

### Uniq
uniq оставляет только уникальные значения. Однако он корректно работает только с отсортированными данными. Для этого мы можем предварительно использовать sort.

Помимо операции схлопывания одинаковых значений, uniq также умеет считать простые статистики для схлопнувшихся групп. Этот функционал чем-то напоминает group by. Так, ключ -c считает количество элементов в каждой группе.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=uniq&category=1&russian=0

In [ ]:
! cat numbers.txt | uniq  # Не совсем тот результат, что мы ожидаем

In [ ]:
! cat numbers.txt | sort | uniq  # А вот так уже работает

По ходу "схлопывания" uniq умеет еще и подсчитывать количество схлопнутых элементов. Таким образом можно считать количество каждого элемента.

In [ ]:
! cat numbers.txt | sort | uniq -c

In [ ]:
%%writefile words.txt
Lorem
ipsum
dolor
sit
met
consectetur
incididunt
elit
seddo
ipsum
tempor
incididunt
ut
laboret
dolor
ipsum
aliqua
ipsum

In [ ]:
! cat words.txt | sort | uniq -c 

### Wc
wc (word count) считает количество элементов во входных данных. По умолчанию считает три характеристики - количество строк, количество слов, количество байт. Различные опции позволяют считать какую-то одну из характеристик. Например -l считает количество строк в данных.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=wc&category=1&russian=0

In [ ]:
! cat numbers.txt | wc

In [ ]:
! cat number-table.txt | wc

In [ ]:
! cat numbers.txt | wc -l # Количество элементов в файле

In [ ]:
! cat numbers.txt | sort | uniq | wc -l  # Количество уникальных элементов в файле

### Cut

cut парсит строки, которые состоят из значений с разделителем. С помощью утилиты можно обрабатывать различные регулярные форматы данных, базирующиеся на разделителях. Например csv или tsv. Для мака: brew install wget

In [ ]:
! wget https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/2/countries.csv

In [ ]:
! cat countries.csv

In [ ]:
! cat countries.csv | cut -d',' -f1,3,5  # разделяем данные по запятой и берем только 6 и 7 столбец

In [ ]:
# удаляем заголовок и смотрим только на Population
! cat countries.csv | tail -n +2 | cut -d',' -f3 | head

In [ ]:
# Считаем уникальные оценки в графе Region
! cat countries.csv | tail -n +2 | cut -d',' -f2 | sort | uniq | wc -l

In [ ]:
! uname -a  # информация о системе

In [ ]:
! uname -a | cut -d" " -f1,3,12  # получаем информацию конкретно про ядро

### Grep

grep позволяет фильтровать входной поток по указанному регулярному выражению

In [ ]:
! head countries.csv

In [ ]:
! cat countries.csv | grep "NORTHERN AFRICA"  # Ищем только NORTHERN AFRICA

флаг -o чтобы оставить только само выражение, -v - все кроме

In [ ]:
! cat countries.csv | grep  -o ".* Island"

In [ ]:
! cat countries.csv | grep -v "EASTERN EUROPE"

### AWK

Программа для обработки структурированного потока со своим собственным небольшим языком программирования.

In [ ]:
! awk 'BEGIN{print "Hello World!"; exit}' 

In [ ]:
! awk '{ if (length($0) > 70) print $0 }' countries.csv

In [ ]:
! cat  countries.csv |  awk -F, '{ if (length($1) > 15) print $2 }'

In [ ]:
! awk -F, '{ if (FNR%15==0) print $0 }' countries.csv

In [ ]:
! cat countries.csv | tail -n +2 | awk -F, 'BEGIN{sum=0.0} {sum+=$3} END{print sum}'

In [ ]:
! cat countries.csv | tail -n +2 | \
awk -F, 'BEGIN{max=0.0; country=""} {if ($4 > max) {max = $4; country=$1}} END{print max,country}' 





In [ ]:
! cat countries.csv | grep Russia

### sed


sed это команда для замен подстрочек. Ей также можно подавать регулярки на вход

In [ ]:
!cat countries.csv | grep "Islands"

In [ ]:
!cat countries.csv | sed "s/Islands/Poopa and Loopa/" | grep "Poopa and Loopa"

In [ ]:
!cat countries.csv | sed "s/.* Islands/Poopa and Loopa/" | grep "Poopa and Loopa"

### Jq
jq не является стандартной программой и ее необходимо самостоятельно установить. Для Ubuntu - apt-get install jq.

jq - это манипулятор JSON документами. Имеет свой язык запросов к JSON, чем то похожий на пайплайны в bash.

Подробнее узнать можно здесь - https://stedolan.github.io/jq/

В файле covid.json содержатся записи о заболеваниях короновирусом в различных странах.

In [ ]:
! wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json

In [ ]:
! jq --help | head

In [ ]:
! head pokedex.json

Основной формат запроса в jq - это json path - путь из ключей, по которому нужно пройтись. Каждый такой path генерирует новый поток данных, согласно этому запросу, который можно дальше модифицировать.

Самый короткий запрос это . - то есть мы запрашиваем весть документ целиком. Попробуем посмотреть на первый элемент в массиве measures.

In [ ]:
! cat pokedex.json | jq '.pokemon[0]'

In [ ]:
! cat pokedex.json | jq '.pokemon[0].weaknesses'

In [ ]:
! cat pokedex.json | jq '. | keys'

In [ ]:
! cat pokedex.json | jq '.pokemon[0] | keys'

In [ ]:
! cat pokedex.json | jq '.pokemon[0].next_evolution[0] | keys'

In [ ]:
! cat pokedex.json | jq '.pokemon[3:5]'

In [ ]:
! cat pokedex.json | jq '.pokemon | length'  # В массиве 151 элемент

In [ ]:
! cat pokedex.json | jq '.pokemon[0:100] | .[].spawn_chance'

In [ ]:
! cat pokedex.json | jq '.pokemon[].candy' | sort | uniq

In [ ]:
! cat pokedex.json | jq -r '.pokemon[].candy' | sort | uniq

С помощью jq можно фильтровать запросы. Для этого есть оператор select

In [ ]:
! cat pokedex.json | jq '.pokemon[] | select(.type[0] == "Water")' | head -n 30

In [ ]:
! cat pokedex.json | jq '.pokemon[] | select(.weaknesses[] | contains("Grass")) | .name'

### Archives
Очень часто данные хранятся в виде архивов. Команды tar и zip\unzip позволяют распаковывать архивы.

tar имеет целый набор однобуквенных ключей, комбинация которых позволяет производить различные операции над архивами.

`c` - создать архив

`x` - распаковать архив

`z` - использовать алгоритм gzip. Архивы, созданные с таким алгоритмом, имеют расширение .tar.gz

`v` - печатать на экран детали распаковки

`f` - считать архив из указанного файла

zip\unzip работает немного проще. Команда zip создает новый архив, команда unzip распаковывает указанный архив.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=tar&category=1 и здесь - https://www.opennet.ru/man.shtml?topic=unzip&category=1&russian=4

In [ ]:
# Создадим архив из файла pokedex
! tar -czvf pokedex.tar.gz pokedex.json

In [ ]:
! head -n 2 pokedex.tar.gz

На самом деле с бинарными файлами также можно работать на ходу: zcat, чтобы вывести, флаг -a для grep, чтобы он не ругался на то, что файл бинарный

In [ ]:
!zcat < pokedex.tar.gz | grep -a "pokemon"

In [ ]:
# Распакуем этот архив в новую директорию tar-grades
! mkdir -p tar-pokemon && tar -xzvf pokedex.tar.gz -C tar-pokemon/

In [ ]:
! ls tar-pokemon/

In [ ]:
! head tar-pokemon/pokedex.json

In [ ]:
# Сделаем точно тоже самое, но с помощью zip\unzip
! zip pokedex.zip pokedex.json

In [ ]:
! head -n 2 pokedex.zip

In [ ]:
! mkdir -p zip-pokemon && unzip pokedex.zip -d zip-pokemon/

In [ ]:
! ls zip-pokemon/

In [ ]:
! head zip-pokemon/pokedex.json

### Networking
wget и curl позволяют выгружать данные из интернета.

wget более продвинутый - он умеет скачивать сразу множество файлов, поддерживает докачку файлов и так далее. curl более простой и может использоваться скорее для точечных запросов.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=wget&category=1&russian=0 и здесь - https://www.opennet.ru/man.shtml?topic=curl&category=1&russian=3



In [ ]:
! mkdir -p nets

In [ ]:
%cd nets

In [ ]:
! curl -L https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json > pokedex.json.curl

In [ ]:
! wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json -O pokedex.json.wget

In [ ]:
! wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json -O - | head

In [ ]:
! ls

Создатим список ссылок и скачаем их все разом с помощью ключа -i



In [ ]:
%%writefile link-list.txt
https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json
https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/2/countries.csv

In [ ]:
! wget -i link-list.txt

In [ ]:
! ls

In [ ]:
! hdfs dfs -mkdir -p /user/pokemons

In [ ]:
! curl https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json | hdfs dfs -put - /user/pokemons/pokedex.json

In [ ]:
! hdfs dfs -ls /user/pokemons

### HDFS

Подключимся к мастер-ноде кластера и попробуем с него поработать с HDFS

```bash
ssh lsml-head

hdfs dfs -ls s3a://lsml-kosmos/
wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json

sudo apt-get update && sudo apt-get install jq -y
cat pokedex.json | jq -r '.pokemon[] | select(.weaknesses[] | contains("Water")) | .name' > water_weak.txt

hdfs dfs -put water_weak.txt s3a://lsml-kosmos/water_weak.txt

hdfs dfs -ls s3a://lsml-kosmos/
hdfs dfs -get s3a://lsml-kosmos/water_weak.txt hdfs_water_weak.txt
```

## Bash scripts

In [ ]:
%%file iterate.json
{
    "a": [1, 2, 3, 4],
    "b": [5, 6, 7, 8]
}

In [ ]:
!python3 a.py

In [ ]:
%%bash
set -ex # e to stop if non-zero exit, x to log every command

exec 1>logs.out # redirecting to files
exec 2>logs.err

config=iterate.json
N_KEYS=$(jq -r '. | keys | length' ${config})
N=$(expr ${N_KEYS} - 1) # setting the correct right border
for i in $(seq 0 ${N})
do 
  key=$(jq -r --argjson idx "$i" '. | keys[$idx]' ${config})
  N_VALUES=$(jq -r --arg key $key '.[$key] | length' ${config})
  M=$(expr ${N_VALUES} - 1)
  for j in $(seq 0 ${M})
  do
      value=$(jq -r --arg market $key --argjson idx "$j" '.[$market] | .[$idx]' ${config})
      echo ${value}
  done
done

In [ ]:
!cat logs.err

In [ ]:
!cat logs.out